In [54]:
import pandas as pd

In [112]:
# Array of CSV files
csv_files = ["../csv/MoveVU_2017_2020.csv", "../csv/MoveVU_2020_2022.csv", "../csv/MoveVU_2022_Oct2023.csv", "../csv/MoveVU_Oct2023_Dec2023.csv"]

df1 = pd.read_csv(csv_files[0], usecols=["ROUTE", "CARDOFFICE_CARD_NUMBER", "CARD_ID_STATUS", "RIDE_DATE", "BUS", "ROUTE", "RUN", "RT_AREA", "RTA_COUNT", "WEGO_COUNT", "FIRST_NAME", "LAST_NAME", "EMPLOYEE_OR_STUDENT", "CAMPUS_ID"])

df2 = pd.read_csv(csv_files[1], usecols=["ROUTE", "CARDOFFICE_CARD_NUMBER", "FIRST_NAME", "LAST_NAME", "EMPLOYEE_OR_STUDENT", "CARD_ID_STATUS", "RIDE_DATE", "BUS", "ROUTE", "RUN", "RT_AREA", "CAMPUS_ID", "RTA_COUNT", "WEGO_COUNT"])

df3 = pd.read_csv(csv_files[2], usecols=["ROUTE", "CARDOFFICE_CARD_NUMBER", "FIRST_NAME", "LAST_NAME", "EMPLOYEE_OR_STUDENT", "CARD_ID_STATUS", "RIDE_DATE", "BUS", "ROUTE", "RUN", "RT_AREA", "CAMPUS_ID", "RTA_COUNT", "WEGO_COUNT"])

df4 = pd.read_csv(csv_files[3], usecols=["ROUTE", "CARDOFFICE_CARD_NUMBER", "FIRST_NAME", "LAST_NAME", "EMPLOYEE_OR_STUDENT", "CARD_ID_STATUS", "RIDE_DATE", "BUS", "ROUTE", "RUN", "RT_AREA", "CAMPUS_ID", "RTA_COUNT", "WEGO_COUNT"])

frames = [df1, df2, df3, df4]
df = pd.concat(frames, ignore_index=True)

# Converting the "RIDE_DATE" columns to datetime objects
df['RIDE_DATE'] = pd.to_datetime(df['RIDE_DATE'])

print("Data Fields:", df.columns.values)
print("Total Number of Vanderbilt Scans in the We Go Bus System:", df.shape[0])

# remove duplicates
df = df.drop_duplicates()

C:\Users\syeda\AppData\Local\Temp\ipykernel_35680\1075164422.py:6: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv(csv_files[1], usecols=["ROUTE", "CARDOFFICE_CARD_NUMBER", "FIRST_NAME", "LAST_NAME", "EMPLOYEE_OR_STUDENT", "CARD_ID_STATUS", "RIDE_DATE", "BUS", "ROUTE", "RUN", "RT_AREA", "CAMPUS_ID", "RTA_COUNT", "WEGO_COUNT"])


Data Fields: ['CARDOFFICE_CARD_NUMBER' 'CARD_ID_STATUS' 'RIDE_DATE' 'BUS' 'ROUTE' 'RUN'
 'RT_AREA' 'RTA_COUNT' 'WEGO_COUNT' 'FIRST_NAME' 'LAST_NAME'
 'EMPLOYEE_OR_STUDENT' 'CAMPUS_ID']
Total Number of Vanderbilt Scans in the We Go Bus System: 1863442


In [116]:
df2023 = df[df['RIDE_DATE'] > '2023-01-01']
print("Total Number of Vanderbilt Scans in the We Go Bus System in 2023:", df2023.shape[0])

dfcolumns_to_check = ['CAMPUS_ID', 'FIRST_NAME', 'LAST_NAME', 'EMPLOYEE_OR_STUDENT']
df2023_cleaned = df2023.dropna(subset=dfcolumns_to_check)

# number of unique CAMPUS_ID
print("Number of unique CAMPUS_ID in 2023:", df2023_cleaned['CAMPUS_ID'].nunique())
# export a csv into the csv folder of the top 1000 riders based upon campus_ID, also include CAMPUS_ID,FIRST_NAME,LAST_NAME,EMPLOYEE_OR_STUDENT. also include the routes they have ridden
# include CAMPUS_ID,FIRST_NAME,LAST_NAME,EMPLOYEE_OR_STUDENT,ROUTE
df2023_cleaned = df2023_cleaned.drop(columns=['RTA_COUNT', 'WEGO_COUNT', 'CARD_ID_STATUS', 'RIDE_DATE', 'CARDOFFICE_CARD_NUMBER', 'RT_AREA', 'RUN', 'BUS'])
# add the count of the number of times a person has ridden the bus
df2023_cleaned['COUNT'] = df2023_cleaned.groupby('CAMPUS_ID')['CAMPUS_ID'].transform('count')

df_unique_routes = df2023_cleaned.groupby('CAMPUS_ID')['ROUTE'].apply(lambda x: sorted([route for route in pd.unique(x) if pd.notnull(route)])).reset_index(name='ROUTES_ARR')

df2023_cleaned = df2023_cleaned.drop(columns=['ROUTE'], errors='ignore')
df2023_with_routes = df2023_cleaned.drop_duplicates(subset=['CAMPUS_ID'])
df2023_with_routes = df2023_cleaned.merge(df_unique_routes, on='CAMPUS_ID', how='left')

# sort the dataframe by the number of times a person has ridden the bus
df2023_with_routes = df2023_with_routes.sort_values(by='COUNT', ascending=False)
df2023.drop_duplicates(subset=['CAMPUS_ID'])
# remove the duplicates from df2023_with_routes
df2023_with_routes = df2023_with_routes.drop_duplicates(subset=['CAMPUS_ID'])
# export to a csv
df2023_with_routes.to_csv('../csv/All_Riders_2023.csv', index=False)

Total Number of Vanderbilt Scans in the We Go Bus System in 2023: 344962
Number of unique CAMPUS_ID in 2023: 5065


In [114]:
df['MONTH_YEAR'] = df['RIDE_DATE'].dt.strftime('%Y-%m')

print("Max Date MM-YY", df['MONTH_YEAR'].max())
print(df['MONTH_YEAR'].unique())

Max Date MM-YY 2023-12
['2019-12' '2019-11' '2019-10' '2019-09' '2019-08' '2019-07' '2019-06'
 '2019-05' '2019-04' '2019-03' '2019-02' '2019-01' '2018-12' '2018-11'
 '2018-10' '2018-09' '2018-08' '2018-07' '2018-06' '2018-05' '2018-04'
 '2018-03' '2018-02' '2018-01' '2017-12' '2017-11' '2017-10' '2017-09'
 '2017-08' '2017-07' '2017-06' '2017-05' '2017-04' '2017-03' '2017-02'
 '2017-01' '2021-12' '2021-11' '2021-10' '2021-09' '2021-08' '2021-07'
 '2021-06' '2021-05' '2021-04' '2021-03' '2021-02' '2021-01' '2020-12'
 '2020-11' '2020-10' '2020-09' '2020-08' '2020-07' '2020-06' '2020-05'
 '2020-04' '2020-03' '2020-02' '2020-01' '2023-09' '2023-08' '2023-07'
 '2023-06' '2023-05' '2023-04' '2023-03' '2023-02' '2023-01' '2022-12'
 '2022-11' '2022-10' '2022-09' '2022-08' '2022-07' '2022-06' '2022-05'
 '2022-04' '2022-03' '2022-02' '2022-01' '2023-12' '2023-11' '2023-10']


In [115]:
top6df = df.drop(['CARDOFFICE_CARD_NUMBER', 'CARD_ID_STATUS', 'RIDE_DATE', 'BUS', 'RUN', 'RT_AREA', 'RTA_COUNT', 'WEGO_COUNT', 'FIRST_NAME', 'LAST_NAME', 'EMPLOYEE_OR_STUDENT', 'CAMPUS_ID'], axis=1)
check_pre_grouping = df[df['MONTH_YEAR'].isin(['2023-10', '2023-11', '2023-12'])]
print("Pre_grouping", check_pre_grouping['MONTH_YEAR'].unique())
# print all of the unique routes during the last 3 months of 2023
print("check_pre_grouping", check_pre_grouping.head(100))

new = top6df.groupby(['MONTH_YEAR', 'ROUTE']).size().reset_index()
# print(new.tail(10))
# print(new['MONTH_YEAR'].unique())
# check_months = new[new['MONTH_YEAR'].isin(['2023-10', '2023-11', '2023-12'])]
# print(check_months['MONTH_YEAR'].unique())

Pre_grouping ['2023-12' '2023-11' '2023-10']
check_pre_grouping         CARDOFFICE_CARD_NUMBER CARD_ID_STATUS                 RIDE_DATE   BUS  \
1773392     393551194=37232=03        UNKNOWN 2023-12-30 23:26:41+00:00  2167   
1773393     730009548=37232=01         ACTIVE 2023-12-30 23:13:47+00:00  1909   
1773394     731553190=37232=01        UNKNOWN 2023-12-30 23:11:47+00:00   726   
1773395     788557875=37232=00        UNKNOWN 2023-12-30 22:41:45+00:00  1914   
1773396     385008873=37232=01       INACTIVE 2023-12-30 22:27:20+00:00  1904   
...                        ...            ...                       ...   ...   
1773487     725553674=37232=00        UNKNOWN 2023-12-30 16:35:03+00:00  2008   
1773488     335557782=37232=02        UNKNOWN 2023-12-30 16:17:49+00:00   137   
1773489     159550780=37232=00        UNKNOWN 2023-12-30 16:17:05+00:00   137   
1773490     516008271=37232=02         ACTIVE 2023-12-30 16:16:47+00:00  1810   
1773491     577000027=37232=05        UNKNOWN